In [ ]:
#r "nuget: Microsoft.Extensions.Configuration.Json,7.0.0"
#r "nuget: Microsoft.Extensions.Configuration.UserSecrets,7.0.0"
#r "nuget: Microsoft.Extensions.Logging.Console,7.0.0"
#r "nuget: Microsoft.Extensions.Logging.Debug,7.0.0"
#r "nuget: Microsoft.SemanticKernel,1.0.0-beta1"
#r "nuget: Microsoft.SemanticKernel.Core, 1.0.0-beta1"
#r "nuget: Microsoft.SemanticKernel.Abstractions, 1.0.0-beta1"
#r "nuget: Microsoft.SemanticKernel.Connectors.AI.OpenAI, 1.0.0-beta1"
#r "nuget: Microsoft.SemanticKernel.Connectors.AI.HuggingFace, 1.0.0-beta1"
#r "nuget: Microsoft.SemanticKernel.Connectors.Memory.Postgres, 1.0.0-beta1"
#r "nuget: Microsoft.SemanticKernel.Connectors.Memory.Sqlite, 1.0.0-beta1"
#r "nuget: Microsoft.SemanticKernel.Connectors.Memory.Weaviate, 1.0.0-beta1"
#r "nuget: Microsoft.SemanticKernel.Connectors.Memory.Qdrant, 1.0.0-beta1"
#r "nuget: Microsoft.SemanticKernel.Functions.Grpc, 1.0.0-beta1"
#r "nuget: Microsoft.SemanticKernel.Functions.OpenAPI, 1.0.0-beta1"
#r "nuget: Microsoft.SemanticKernel.Functions.Semantic, 1.0.0-beta1"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.0.0-beta1"
#r "nuget: Microsoft.SemanticKernel.Plugins.Memory, 1.0.0-beta1"
#r "nuget: Microsoft.SemanticKernel.Plugins.Document, 1.0.0-beta1"
#r "nuget: Microsoft.SemanticKernel.Planners.Core, 1.0.0-beta1"
#r "nuget: Microsoft.SemanticKernel.Plugins.Web, 1.0.0-beta1"
#r "nuget: Microsoft.SemanticKernel.TemplateEngine.Basic, 1.0.0-beta1"
#r "nuget: System.CommandLine,2.0.0-beta4.22272.1"
#r "nuget: YamlDotNet,13.5.2"

In [ ]:
using System;
using System.IO;
using System.CommandLine;
using System.Collections.Immutable;
using System.Text.RegularExpressions;
using Microsoft.Extensions.Configuration;
using Microsoft.Extensions.Logging;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.AI.TextCompletion;
using Microsoft.SemanticKernel.Connectors.AI.OpenAI.TextEmbedding;
using Microsoft.SemanticKernel.Connectors.Memory.Weaviate;
using Microsoft.SemanticKernel.Connectors.Memory.Qdrant;
using Microsoft.SemanticKernel.Connectors.Memory.Postgres;
using Microsoft.SemanticKernel.Connectors.Memory.Sqlite;
using Microsoft.SemanticKernel.Events;
using Microsoft.SemanticKernel.Functions.OpenAPI;
using Microsoft.SemanticKernel.Memory;
using Microsoft.SemanticKernel.Orchestration;
using Microsoft.SemanticKernel.Planning;
using Microsoft.SemanticKernel.Plugins.Core;
using Microsoft.SemanticKernel.Plugins.Memory;
using Microsoft.SemanticKernel.SemanticFunctions;
using Microsoft.SemanticKernel.TemplateEngine.Basic;

In [14]:
var notebookDir = Environment.CurrentDirectory;
var parentDir = Directory.GetParent(notebookDir).FullName;

var builder = new ConfigurationBuilder()
    .SetBasePath(parentDir)  // Set the base path
    .AddJsonFile("config/appsettings.json", optional: false, reloadOnChange: true)  // Load the main appsettings.json file
    .AddJsonFile("appsettings.plugins.json", optional: true, reloadOnChange: true);  // Load the environment-specific appsettings file

var conf = builder.Build();

ILoggerFactory loggerFactory = LoggerFactory.Create(builder =>
{
  builder.SetMinimumLevel(LogLevel.Warning)
          .AddConsole()
          .AddDebug();
});

var serviceId = conf.GetValue<string>("serviceId");
var endpointType = conf.GetValue<string>("endpointType");
var serviceType = conf.GetValue<string>("serviceType");
var deploymentOrModelId = conf.GetValue<string>("deploymentOrModelId");
var apiKey = conf.GetValue<string>("apiKey");
var orgId = conf.GetValue<string>("orgId");

// Taken from: https://github.com/microsoft/semantic-kernel/blob/main/dotnet/samples/KernelSyntaxExamples/Example42_KernelBuilder.cs
//
// Kernel.Builder returns a new builder instance, in case you want to configure the builder differently.
// ==========================================================================================================
// Kernel instances can be created the usual way with "new", though the process requires particular
// attention to how dependencies are wired together. Although the building blocks are available
// to enable custom configurations, we highly recommend using KernelBuilder instead, to ensure
// a correct dependency injection.
// ----------------------------------------------------------------------------------------------------------
IKernel kernel = Kernel.Builder
                     .WithLoggerFactory(loggerFactory)
                     .WithOpenAITextCompletionService(
                            modelId: deploymentOrModelId,
                            apiKey: apiKey,
                            orgId: orgId,
                            serviceId: serviceId)
                    .Build();

// var memoryStore = new QdrantMemoryStore("http://localhost:6333", 1536, loggerFactory: loggerFactory);
var memoryStore = await SqliteMemoryStore.ConnectAsync("aboutme.sqlite");

// Create an embedding generator to use for semantic memory.
var embeddingGenerator = new OpenAITextEmbeddingGeneration(modelId: "text-embedding-ada-002", apiKey: apiKey,
                                                                organization: orgId,
                                                                loggerFactory: loggerFactory);


In [15]:
var rootDirectory = conf.GetSection("PluginSettings:Root").Get<string>();
var pluginDirectories = conf.GetSection("PluginSettings:Plugins").Get<string[]>();

var pluginsRoot = Path.Combine(parentDir, rootDirectory!);
var pluginImport = kernel.ImportSemanticFunctionsFromDirectory(pluginsRoot, pluginDirectories!);

In [16]:
int nameMaxLength = pluginImport.Max(p => p.Value.Name.Length) + 2;  // +2 for padding
int descMaxLength = pluginImport.Max(p => p.Value.Description.Length) + 2;

// Header
Console.WriteLine($"+{new string('-', nameMaxLength)}+{new string('-', descMaxLength)}+");
Console.WriteLine($"|{"Plugin".PadRight(nameMaxLength)}|{"Description".PadRight(descMaxLength)}|");
Console.WriteLine($"+{new string('-', nameMaxLength)}+{new string('-', descMaxLength)}+");

// Rows
foreach (var p in pluginImport)
{
    Console.WriteLine($"|{p.Value.Name.PadRight(nameMaxLength)}|{p.Value.Description.PadRight(descMaxLength)}|");
}

// Footer
Console.WriteLine($"+{new string('-', nameMaxLength)}+{new string('-', descMaxLength)}+");

+---------------+----------------------------------------------------+
|Plugin         |Description                                         |
+---------------+----------------------------------------------------+
|Helm           |Create Helm Charts for given devops task            |
|Kubernetes     |Create kubernetes YAML files for given devops task  |
|CSharp         |Create .NET projects for given task                 |
|TypeScript     |Create TypeScript source codes for given task       |
|CreateHtmlDoc  |Create HTML documents                               |
|ExtractJS      |Extract JavaScript from an HTML document            |
|UrlFinder      |Find the URL of a web page                          |
|Chat           |Answer user questions                               |
+---------------+----------------------------------------------------+


In [17]:

// The combination of the text embedding generator and the memory store makes up the 'SemanticTextMemory' object used to
// store and retrieve memories.
var textMemory = new MemoryBuilder()
            .WithLoggerFactory(loggerFactory)
            .WithTextEmbeddingGeneration(embeddingGenerator)
            .WithMemoryStore(memoryStore)
            .Build();

bool exists = await memoryStore.DoesCollectionExistAsync("aboutme");

if (!exists) {
  await memoryStore.CreateCollectionAsync("aboutme");
}

// Alternatively, one could use SemanticTextMemory instance instead
//SemanticTextMemory textMemory = new(memoryStore, embeddingGenerator);

In [18]:
// Helper class for our facts
public class Fact
{
  public string Text { get; }
  public string Id { get; } = Guid.NewGuid().ToString();
  public string Description { get; }
  public string AdditionalMetadata { get; }

  public Fact(string text, string description, string additionalMetadata)
  {
    Text = text;
    Description = description;
    AdditionalMetadata = additionalMetadata;
  }
}

var facts = new Fact[]
    {
        new("I was born in Berlin.", "Place of birth", "City: Berlin"),
        new("I am 25 years old.", "Age", "Years: 25"),
        new("My favorite sports team is Lakers.", "Favorite sports team", "Team: Lakers"),
        new("I have 2 siblings.", "Siblings", "Number: 2"),
        new("I work as a developer.", "Occupation", "Job Title: Developer"),
        new("I enjoy hiking.", "Hobbies", "Activity: Hiking"),
        new("I have a pet dog.", "Pets", "Type: Dog"),
        new("My favorite cuisine is Italian.", "Favorite Cuisine", "Cuisine: Italian"),
        new("I have visited 5 countries.", "Travel", "Countries: 5"),
        new("I graduated from the University of London.", "Education", "University: London"),
        new("I speak 3 languages.", "Languages Spoken", "Number: 3"),
        new("I am allergic to peanuts.", "Allergies", "Allergen: Peanuts"),
        new("I have run a marathon.", "Athletic Achievements", "Event: Marathon"),
        new("I have a collection of vintage stamps.", "Collections", "Item: Stamps"),
        new("I prefer autumn over other seasons.", "Seasonal Preferences", "Season: Autumn"),
        new("My favorite book is 'To Kill a Mockingbird'.", "Favorite Book", "Book: To Kill a Mockingbird"),
        new("I am a vegetarian.", "Diet", "Diet: Vegetarian"),
        new("I have volunteered at a local shelter.", "Volunteering", "Place: Local Shelter"),
        new("I have a goal to visit every continent.", "Life Goals", "Goal: Visit Every Continent"),
        new("I play the guitar.", "Musical Instruments", "Instrument: Guitar"),
        new("I have a master's degree in Computer Science.", "Advanced Education", "Degree: Master's in Computer Science")
    };

In [19]:
var memoryPlugin = new TextMemoryPlugin(textMemory);
var memoryFunctions = kernel.ImportFunctions(memoryPlugin, "MemoryPlugin");

var collection = "aboutme";

// use this to wipe memory
// await memoryStore.DeleteCollectionAsync(collection);

bool exists = await memoryStore.DoesCollectionExistAsync(collection);

if (!exists)
{
  // there are two ways to populate the memory:
  // * with Kernel + Plugins
  var results = new List<KernelResult?>();
  foreach (var fact in facts)
  {
    var result = await kernel.RunAsync(memoryFunctions["Save"], new()
    {
      [TextMemoryPlugin.CollectionParam] = collection,
      [TextMemoryPlugin.KeyParam] = fact.Id,
      ["input"] = fact.Text
    });
    results.Add(result);
  }
 
  // * with SemanticTextMemory
  // var ids = new List<string>();
  // foreach (var fact in facts)
  // {
  //   var id = await memory.SaveInformationAsync(
  //     collection: collection,
  //     text: fact.Text,
  //     id: fact.Id,
  //     description: fact.Description,
  //     additionalMetadata: fact.AdditionalMetadata);
  //   ids.Add(id);
  // }
}

In [20]:
    // * There are three ways to query the memory

    // * 1. By using TextMemory methods
    // MemoryQueryResult? lookup = await textMemory.GetAsync(collection, "INSERT_ID_HERE");
    IAsyncEnumerable<MemoryQueryResult> lookup = textMemory.SearchAsync(collection, "Do I have pets?", 1, minRelevanceScore: 0.50);

    await foreach (var r in lookup)
    {
      Console.WriteLine(r.Metadata.Text ?? "ERROR: memory not found");
    }

I have a pet dog.


In [21]:
// * 2. By using Kernel + Plugins
var result = await kernel.RunAsync(memoryFunctions["Recall"], new()
{
  [TextMemoryPlugin.CollectionParam] = collection,
  [TextMemoryPlugin.LimitParam] = "1",
  [TextMemoryPlugin.RelevanceParam] = "0.79",
  ["input"] = "What is my age?"
});
Console.WriteLine($"Answer: {result.GetValue<string>()}");

Answer: I am 25 years old.


In [25]:
// * 3. By using the RAG Pattern (Retrieval Augmented Generation)
static async Task<string> RunMiniRAG(ISemanticTextMemory memory, IKernel kernel, string collectionName, string input)
{
  var context = kernel.CreateNewContext();
  context.Variables.Add("user_input", input);

  // retrieve user-specific context based on the user input
  var searchResults = memory.SearchAsync(collectionName, input);
  var retrieved = new List<string>();
  await foreach (var item in searchResults)
  {
    retrieved.Add(item.Metadata.Text);
  }
  context.Variables.Add("user_context", string.Join(',', retrieved));

  // run SK function and give it the two variables, input and context
  var func = kernel.Functions.GetFunction("Assistant", "Chat");
  var answer = await kernel.RunAsync(func, context.Variables);

  return answer.GetValue<string>()!;

}

var answer = await RunMiniRAG(textMemory, kernel, collection, "Tell me something about my hobbies.");
Console.WriteLine(answer);


You seem to really enjoy hiking, and it's a great way to stay active and explore the outdoors. It's also a great way to get some fresh air and clear your head.
